In [1]:
import os
import json
import time

import numpy as np
import torch
from peft import PeftModel
from transformers import AutoTokenizer, LlamaForCausalLM

from generation.utils import GenerationMixin
from tone_utils import find_tone


In [2]:
# Auto model won't work here, you will need to change the class name for a new architecture
# Auto model use some black magic fuckery to determine the correct class name which will ignore our custom mixin
class CustomLlamaForCausalLM(GenerationMixin, LlamaForCausalLM):
    pass

model_name = 'bkai-foundation-models/vietnamese-llama2-7b-40GB'
token = ""
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True,
                                          token=token)

model = CustomLlamaForCausalLM.from_pretrained(model_name, 
                                                # load_in_8bit=True,
                                                token=token,
                                                device_map={'': 0},
                                                torch_dtype=torch.bfloat16)
model = PeftModel.from_pretrained(model, "suicaokhoailang/bkllama2-poem-generator")

In [3]:
rhymes = json.load(open("./data/rhymables.json"))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Chỉ dùng các âm tiết xác định được vần điệu
allowed_ids = [x['id'] for x in rhymes.values()] + tokenizer.encode("\n")[-1:]
all_ids = np.arange(len(tokenizer.get_vocab()))
not_allowed_ids = set(all_ids) - set(allowed_ids)
not_allowed_ids = [[x] for x in not_allowed_ids]

# Từ điển vần
rhymable_dict = dict([(val['id'], set(val["rhymes_with_ids"])) for key, val in rhymes.items()])
# Các âm bằng trắc
even_ids = [val['id'] for key, val in rhymes.items() if find_tone(key) < 2]
uneven_ids = [val['id'] for key, val in rhymes.items() if find_tone(key) >= 2]
# Các âm huyền, ngang
unmarked_ids = [val['id'] for key, val in rhymes.items() if find_tone(key) == 0]
grave_ids = [val['id'] for key, val in rhymes.items() if find_tone(key) == 1]

In [5]:
def generate_prompt(text):
    return f""" 
Viết bài thơ có nội dung: {text}
### Trả lời:
"""[1:]

def generate(text, max_pairs=5):
    text = generate_prompt(text)
    input_ids = tokenizer.encode(text, return_tensors='pt').to(device)
    min_length = 16 * 2 + len(input_ids[0])
    max_length = 16 * max(3,max_pairs) + len(input_ids[0])  # + 40
    sample_outputs = model.generate(input_ids=input_ids, pad_token_id=tokenizer.eos_token_id,
                                    do_sample=False,
                                    max_length=max_length,
                                    min_length=min_length,
                                    num_beams=5,
                                    no_repeat_ngram_size=3,
                                    num_return_sequences=1,
                                    eos_token_id=[-1],
                                    bad_words_ids=not_allowed_ids,
                                    newline_id=tokenizer.encode("\n")[-1:],
                                    repetition_penalty=1.5,
                                    temperature=1.5,
                                    all_ids=all_ids,
                                    rhymable_dict=rhymable_dict,
                                    grave_ids=grave_ids,
                                    unmarked_ids=unmarked_ids,
                                    even_ids=even_ids,
                                    uneven_ids=uneven_ids,
                                    generate_6_8=True)

    return tokenizer.decode(sample_outputs[0].tolist(), skip_special_tokens=False)

In [6]:
text = 'ăn phở'
print(generate(text,max_pairs=5))


<s> 
Viết bài thơ có nội dung: ăn phở
### Trả lời:
ăn gì ngon miệng không chê
chỉ chê người bán dở nghề thôi nha
ăn rồi lại thấy nhớ nhà
ăn thêm một bát nữa nha nó về
ăn hoài vẫn thấy đói ghê
ăn bao nhiêu vẫn chưa về nhà đâu
ăn nhiều lại thấy buồn rầu
ăn càng nhiều lại càng sầu khổ đau
ăn cho hết sạch còn đâu
cũng đành bỏ lại ở sau quán này
